In [1]:
import pandas as pd
import lightfm
from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k, reciprocal_rank, auc_score
from lightfm.cross_validation import random_train_test_split
from dataloading import DataLoader
from datapreparation_py import DataPreparation
from lightFM_adoptor import LightFMAdaptor
from cv_py import CrossValidation
from implicit.evaluation import ranking_metrics_at_k, train_test_split

c:\Users\781110104\Anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [2]:
%cd C:\Users\781110104\OneDrive - Genpact\Documents\VSCode

C:\Users\781110104\OneDrive - Genpact\Documents\VSCode


In [3]:
dl = DataLoader()
user_item_co_t = dl.import_data('TEREX', 'CO', 'df')

In [4]:
user_item_co_t_filtered = dl.remove_low_interact_items(user_item_co_t, 1)

In [5]:
dp = DataPreparation(user_item_co_t_filtered)

In [6]:
interactions_csr, weights_csr = dp.get_interaction_data()

In [7]:
user_features = ['country']
item_features = ['group2']
feature_input = dp.get_feature_data(user_features=user_features, item_features=item_features)

In [8]:
userf, itemf = feature_input

In [9]:
train_i, test_i = train_test_split(interactions_csr, 0.8, 22)
train_w, test_w = train_test_split(weights_csr, 0.8, 22)

In [10]:
cv = CrossValidation(5)

In [11]:
val_w, cal_w = cv.split_k_fold(train_w, 22)

Testing only the model:

In [20]:
model_adapt = LightFMAdaptor(loss='warp', random_state=22, no_components=5, iterations=1)

In [18]:
model_adapt.fit(interactions=train_i, user_features=userf, item_features=itemf, weights=train_w, show_progress=True)

Epoch: 100%|██████████| 1/1 [00:00<00:00,  5.93it/s]


In [19]:
ranking_metrics_at_k(model_adapt, train_w, test_w, 10)

  0%|          | 0/40 [00:00<?, ?it/s]

{'precision': 0.1125,
 'map': 0.05124206349206349,
 'ndcg': 0.11226154526810625,
 'auc': 0.5016779690524178}

Testing the model with simple parameter tuning:

In [15]:
space_FM_simple = {'factors' : [5], 'regularization' : [0], 'learning_rate' : [0.05], 'iterations' : [1], 'loss' : ['warp'], 'max_sampled' : [10]}
hyper_fm_simple = cv.hyperp_tuning_simple(test=test_w, train=train_w, seed=22, param_space=space_FM_simple, model_class='FM', 
user_features=userf, item_features=itemf)

In [16]:
hyper_fm_simple

,factors,regularization,learning_rate,iterations,loss,max_sampled,precision,map,ndcg,auc,mpr
0,5,0,0.05,1,warp,10,0.1125,0.051242,0.112262,0.501678,0.12872


Testing the model with k-fold crossvalidation parameter tuning:

In [15]:
space_FM = {'factors' : [5], 'regularization' : [0], 'learning_rate' : [0.05], 'iterations' : [1], 'loss' : ['warp'], 'max_sampled' : [10]}
hyper_fm = cv.hyperp_tuning(test=val_w, train=cal_w, exclude=test_w, seed=22, param_space=space_FM, model_class='FM', 
user_features=userf, item_features=itemf)

In [16]:
hyper_fm

,factors,regularization,learning_rate,iterations,loss,max_sampled,precision,map,ndcg,auc,mpr
0,5,0,0.05,1,warp,10,0.1155,0.048308,0.114431,0.502374,0.151519
